## Example Usage of hec.unit Module ##

### Setup ###

In [19]:
import os, sys

srcdir = os.path.join(os.path.dirname(os.getcwd()), "src")
sys.path.append(srcdir)
from hec import unit  # type: ignore

### Get Some Unit Names

In [20]:
print(f"length units  = {unit.get_compatible_units('foot')}")
print(f"area units    = {unit.get_compatible_units('foot**2')}")
print(f"volumne units = {unit.get_compatible_units('foot**3')}")
print(f"flow units    = {unit.get_compatible_units('ft**3/s')}")

length units  = ['cm', 'ft', 'ftUS', 'in', 'km', 'm', 'mi', 'mm']
area units    = ['1000 m2', 'acre', 'cm2', 'ft2', 'ha', 'km2', 'm2', 'mile2']
volumne units = ['1000 m3', 'ac-ft', 'dsf', 'ft3', 'gal', 'kaf', 'kdsf', 'kgal', 'km3', 'm3', 'mcm', 'mgal', 'mile3']
flow units    = ['cfs', 'cms', 'gpm', 'KAF/mon', 'kcfs', 'kcms', 'mcm/mon', 'mgd']


### Get Aliases For a Unit

In [21]:
print(f"Aliases for ft  = {unit.get_unit_aliases('ft')}")
print(f"Aliases for ft2 = {unit.get_unit_aliases('ft2')}")
print(f"Aliases for ft3 = {unit.get_unit_aliases('ft3')}")
print(f"Aliases for cfs = {unit.get_unit_aliases('cfs')}")

Aliases for ft  = ['FEET', 'FT', 'Feet', 'feet', 'foot']
Aliases for ft2 = ['sq ft', 'square feet']
Aliases for ft3 = ['cu ft', 'cubic feet']
Aliases for cfs = ['CFS', 'FT3/S', 'FT3/SEC', 'cu-ft/sec', 'cuft/sec', 'cusecs', 'ft3/s', 'ft3/sec', 'ft^3/s']


### Simple Unit Conversions

In [23]:
print(f"1 mile = {unit.convert_units(1, 'mi', 'ft')} feet")
print(f"1 hectare = {unit.convert_units(1, 'ha', 'acre',)} acres")

1 mile = 5280.0 feet
1 hectare = 2.4710538146716536 acres


### Using Pint

In [33]:
ureg = unit.get_unit_registry()
ctx = unit.get_unit_context()
u_acre = unit.get_pint_unit("acre")
u_ft = unit.get_pint_unit("ft")
u_mgal = unit.get_pint_unit("mgal")
area = 10 * u_acre
depth = 50 * u_ft
vol = area * depth
vol_mgal = (area * depth).to(u_mgal, ctx)
print(f"{area:~P} * {depth:~P} = {vol:~P}")
print(f"{area:~P} * {depth:~P} = {vol_mgal:~P}")
print(f"{area:~P} * {depth:~P} = {vol_mgal.magnitude} {unit.get_unit_name(u_mgal)}")
print(
    f"10 acres * 50 feet = {(ureg('10 acres') * ureg('50 feet')).to(ureg('Mgal'), ctx)}"
)

10 acre * 50 ft = 500 acre·ft
10 acre * 50 ft = 162.9257142857143 Mgal
10 acre * 50 ft = 162.9257142857143 mgal
10 acres * 50 feet = 162.9257142857143 megagallon
